In [1]:
import os
import MDAnalysis as mda
from MDAnalysis.analysis import psa
import plotly.graph_objects as go
import mdtraj as md
from tqdm import tqdm

In [8]:
trajectory_file = 'I:/Nick/200ns/100ns/2/md_0_1.xtc'
topology_file = 'I:/Nick/200ns/100ns/2/EKB1m.pdb'
output_dir = 'I:/Nick/200ns/100ns/2'
chunk_size_ns = 10

In [9]:
traj = md.load(trajectory_file, top=topology_file)
total_frames = traj.n_frames
frame_rate = traj.timestep

OSError: No such file: I:/Nick/200ns/100ns/2/md_0_1.xtc

In [ ]:
chunk_size_frames = int(chunk_size_ns / frame_rate)

for chunk_start in range(0, total_frames, chunk_size_frames):
    chunk_end = min(chunk_start + chunk_size_frames, total_frames)
    chunk = traj[chunk_start:chunk_end]
    output_file = f'{output_dir}/chunk_{chunk_start}-{chunk_end}.xtc'
    chunk.save(output_file)


In [ ]:
import plotly.subplots as sp
# ANOVA HEATMAP
fig = sp.make_subplots(rows=5, cols=4, subplot_titles=[f"df{i}" for i in range(1,21)],
                       shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.03, horizontal_spacing=0.03,
                       column_width=[0.8]*4, row_heights=[0.2, 0.2, 0.2, 0.2, 0.2], specs=[[{'type': 'heatmap'}]*4]*5,
                       print_grid=False, )
fig.update_layout(height=1000, width=1200, title='Significance of differences in means', font=dict(size=10))

for i in range(1, 21):
    results = []
    for j in range(1, 21):
        row = []
        for c in range(0, 15):
            f_value, p_value = stats.f_oneway(dfs[j-1].iloc[:, c], dfs[i-1].iloc[:, c])
            if p_value < 0.05:
                row.append(1)
            else:
                row.append(0)
        results.append(row)
    results = np.array(results)
    percentage = 100 * np.count_nonzero(results) / results.size
    print(f"Percentage of significant's over the total for df{i}: {percentage:.2f}%")

    heatmap = go.Heatmap(z=results, x=[i for i in range(1,16)], y=[(j)*10 for j in range(1,21)], colorscale='Viridis')
    row = (i-1) // 4 + 1
    col = (i-1) % 4 + 1
    fig.add_trace(heatmap, row=row, col=col)

fig.update_xaxes(title_text='Variant', row=5, col=3, title_font=dict(size=20), title_standoff=2.5)
fig.update_yaxes(title_text='Duration (ns)', row=3, col=1, title_font=dict(size=20))

fig.show()


In [15]:
def run_psa(folder_path):
    # Get a list of all .xtc files in the folder
    traj_files = [f for f in os.listdir(folder_path) if f.endswith('.xtc')]

    # Find the PDB file in the same directory as the trajectory files
    pdb_file = None
    for f in os.listdir(folder_path):
        if f.endswith('.pdb'):
            pdb_file = os.path.join(folder_path, f)
            break

    if pdb_file is None:
        print(f'No PDB file found in {folder_path}')
        return

    # Create a list of universe objects for each trajectory file
    trajectories = []
    labels = []
    for traj_file in traj_files:
        traj_path = os.path.join(folder_path, traj_file)
        label = os.path.splitext(traj_file)[0].split('_')[-1]
        labels.append(label)
        u = mda.Universe(pdb_file, traj_path)
        trajectories.append(u)

    # Create a reference universe object from the PDB file
    ref = mda.Universe(pdb_file)

    # Initialize a PSA object with the trajectories, labels, and reference
    ps = psa.PSAnalysis(trajectories,
                        labels=labels,
                        reference=ref,
                        select='name CA',
                        path_select='name CA')

    # Run the Hausdorff distance metric
    ps.run(metric='hausdorff')

    # Get the array map from the PSA object
    array_map = ps.get_output()

    # Create the figure and add the heatmap trace
    fig = go.Figure(data=go.Heatmap(z=np.array(array_map), colorscale='RdYlBu', showscale=True, reversescale=False))

    # Set the x-axis and y-axis labels
    fig.update_layout(xaxis_title='Simulation Chunk', yaxis_title='Simulation Chunk')

    # Save the plot with the relevant name
    plot_name = f'Hausdorff_{os.path.basename(folder_path)}'
    fig.write_html(f'{plot_name}.html')

def run_psa_on_all_directories(root_folder):
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for dirname in dirnames:
            folder_path = os.path.join(dirpath, dirname)
            run_psa(folder_path)


In [16]:
root_folder = '/mnt/f/200ns/100ns.2'
run_psa_on_all_directories(root_folder)